In [349]:
!pip install neo4j    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [350]:
from neo4j import GraphDatabase 

In [351]:
uri = "neo4j+s://70bc3204.databases.neo4j.io"
pwd = "7de6kd6oEFEU8wwTlLbOI3F9cP6NSg2Ibxsy4zOcwZE"
driver = GraphDatabase.driver(uri, auth=("neo4j", pwd))    

In [352]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [353]:
uri = "neo4j+s://70bc3204.databases.neo4j.io"
pwd = "7de6kd6oEFEU8wwTlLbOI3F9cP6NSg2Ibxsy4zOcwZE"
user = 'neo4j'
conn = Neo4jConnection(uri, user, pwd)

## **CONSTRAINTS**

In [354]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CVE IF NOT EXISTS FOR (cve:CVE) REQUIRE cve.id IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [355]:
explore_reviews_csv_query = """
CREATE CONSTRAINT references IF NOT EXISTS FOR (ref:Reference) REQUIRE ref.url IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [356]:
explore_reviews_csv_query = """
CREATE CONSTRAINT descriptions IF NOT EXISTS FOR (desc:Description) REQUIRE desc.id IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [357]:
explore_reviews_csv_query = """
CREATE CONSTRAINT tags IF NOT EXISTS FOR (t:Tag) REQUIRE t.tag IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [358]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CONFIG IF NOT EXISTS FOR (config:CONFIG) REQUIRE config.id IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [359]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CPE IF NOT EXISTS FOR (cpe:CPE) REQUIRE cpe.cpe23Uri IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [360]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CWE IF NOT EXISTS FOR (cwe:CWE) REQUIRE cwe.CWE IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [361]:
explore_reviews_csv_query = """
CREATE CONSTRAINT CPE IF NOT EXISTS FOR (cpe:CPE) REQUIRE cpe.CPE IS UNIQUE
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **CVE Data**

**Script_1.cypher**

In [362]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/nvd3.json", null, {failOnError:false})
YIELD value
WITH value  AS data

UNWIND data.CVE_Items as item
MERGE (cve:CVE {id: item.cve.CVE_data_meta.ID})
ON CREATE SET cve.assigner = item.cve.CVE_data_meta.ASSIGNER

WITH cve, item
UNWIND item.cve.problemtype.problemtype_data AS problem
UNWIND problem.description AS problem_desc
MERGE (cve)-[:PROBLEM_TYPE]->(problemType:ProblemType {lang: problem_desc.lang, value: problem_desc.value})
//MERGE (cve)-[:PROBLEM_TYPE]->(problemType)


WITH item, cve
UNWIND item.cve.references.reference_data AS ref
MERGE (reference:Reference {url: ref.url})
ON CREATE SET reference.name = ref.name, reference.refsource = ref.refsource

WITH item, cve, reference, ref
UNWIND ref.tags AS tag
MERGE (tagNode:Tag {name: tag, id: tag})
MERGE (reference)-[:TAGGED_WITH]->(tagNode)

WITH item, cve, reference
UNWIND item.cve.description.description_data AS desc
MERGE (description:Description {lang: desc.lang, value:desc.value, cveId:cve.id})
//ON CREATE SET description.value = desc.value

//WITH item, cve, reference, description
MERGE (cve)-[:HAS_REFERENCE]->(reference)
MERGE (cve)-[:HAS_DESCRIPTION]->(description)



WITH item, cve,item.impact.baseMetricV3 as baseMetricV3, item.impact.baseMetricV3.cvssV3 as cvssV3
MERGE (cve)-[:HAS_IMPACT]->(impact:IMPACT {exploitabilityScore: baseMetricV3.exploitabilityScore, 
                                           impactScore: baseMetricV3.impactScore, cveId: cve.id
})-[:HAS_CVSSV3]->(cvssv3:CVSSV3 {version: cvssV3.version,
                                              vectorString:cvssV3.vectorString,
                                              attackVector:cvssV3.attackVector,
                                              attackComplexity:cvssV3.attackComplexity,
                                              privilegesRequired:cvssV3.privilegesRequired,
                                              userInteraction:cvssV3.userInteraction,
                                              scope:cvssV3.scope,
                                              confidentialityImpact:cvssV3.confidentialityImpact,
                                              integrityImpact:cvssV3.integrityImpact,
                                              availabilityImpact:cvssV3.availabilityImpact,
                                              baseScore:cvssV3.baseScore,
                                              baseSeverity:cvssV3.baseSeverity 
})
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


**Script_2.cypher**

In [363]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/nvd3.json", null, {failOnError:false})
YIELD value
WITH value  AS data

UNWIND data.CVE_Items as item
MERGE (cve:CVE {id: item.cve.CVE_data_meta.ID})
ON CREATE SET cve.assigner = item.cve.CVE_data_meta.ASSIGNER

WITH item.configurations AS configurations, cve
MERGE (config:Config {version: configurations.CVE_data_version, cveId: cve.id})
MERGE (cve)-[:HAS_CONFIG]->(config)

WITH configurations, config, cve
UNWIND configurations.nodes as node
  // Each node has operator, cpe_match and  children
  //create operator for the node
  CREATE (op:Operator {operator: node.operator})
  CREATE (config)-[:HAS_OPERATOR]->(op)
  //create cpe_match for the node
  FOREACH (cpe_match in node.cpe_match |
    CREATE (cpeMatch:CpeMatch {vulnerable: cpe_match.vulnerable, uri: cpe_match.cpe23Uri})
    CREATE (cpeMatch)-[:HAS_OPERATOR]->(op))

    //children in each node -  each children then has operator, cpe_match and children
    FOREACH (child IN node.children | 
      CREATE (op1:Operator {operator: child.operator})
      CREATE (op)-[:HAS_OPERATOR]->(op1)
      FOREACH (cpe in child.cpe_match |
        CREATE (cpeMatch:CpeMatch {vulnerable: cpe.vulnerable, uri: cpe.cpe23Uri})
        MERGE (op1)-[:CPE_MATCH]->(cpeMatch))
)
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **CWE Data**

**Script_CWE.cypher**

In [364]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/CWE.json", null, {failOnError:false})
YIELD value
WITH value  AS data
//return data.CWE_Items[0].CWE.cwe_id
UNWIND data.CWE_Items as item
MERGE (cwe:CWE {cwe_id: item.CWE.cwe_id})
SET 
    cwe.name = item.CWE.name,
    cwe.weakness_abstraction = item.CWE.weakness_abstraction, 
    cwe.status = item.CWE.status, 
    cwe.description = item.CWE.description,  
    cwe.extended_description = item.CWE.extended_description

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


Mapping between CVE and CWE

In [365]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-79"})
MERGE (cwe:CWE {cwe_id:"79"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [366]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-89"})
MERGE (cwe:CWE {cwe_id:"89"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [367]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-94"})
MERGE (cwe:CWE {cwe_id:"94"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [368]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-200"})
MERGE (cwe:CWE {cwe_id:"200"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [369]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-284"})
MERGE (cwe:CWE {cwe_id:"284"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [370]:
explore_reviews_csv_query = """
MERGE (problemType:ProblemType {value:"CWE-294"})
MERGE (cwe:CWE {cwe_id:"294"})
MERGE (problemType)-[:MAPPED_WITH]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [371]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"79"})
MERGE (cwe2:CWE { cwe_id:"89"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1003",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [372]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"79"})
MERGE (cwe2:CWE { cwe_id:"94"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1000",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [373]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"79"})
MERGE (cwe2:CWE { cwe_id:"94"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1003",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [374]:
explore_reviews_csv_query = """
MERGE (cwe1:CWE { cwe_id:"89"})
MERGE (cwe2:CWE { cwe_id:"94"})
MERGE (cwe1)-[:HAS_RELATED_WEAKNESS{ NATURE:"ChildOf",CWEـID:"74",VIEWـID:"1003",ORDINAL:"Primary"}]->(cwe2)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


Extract the entities from unstructured data (extended_description) in CWE to structured data by using entity-fishing

In [375]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/Extended_Descriptions.json", null, {failOnError:false})
YIELD value
WITH value  AS data
//return data.Extended_Descriptions[0].CWE.runtime
UNWIND data.Extended_Descriptions as extended_descript
MERGE (cwedata:Extended_Descriptions {runtime: extended_descript.CWE.runtime})
SET 
    cwedata.software = extended_descript.CWE.software,
    cwedata.version = extended_descript.CWE.version,
    cwedata.date = extended_descript.CWE.date,
    cwedata.nbest = extended_descript.CWE.nbest,
    cwedata.text = extended_descript.CWE.text

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [376]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"79"})
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.04878159541353045,source:"wikipedia-en",category:"Hacking (computer security)",page_id:11647669}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.026132997553802943,source:"wikipedia-en",category:"Web security exploits",page_id:5131853}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.001742199837897568,source:"wikipedia-en",category:"Unicode",page_id:720904}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [377]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/Extended_Descriptions.json", null, {failOnError:false})
YIELD value
WITH value  AS data
WITH data
UNWIND data.Extended_Descriptions[0].CWE.entities as Entities
MERGE (entities:Entities{rawName:Entities.`rawName`})
  SET 
   entities.offsetStart = Entities.offsetStart,
   entities.offsetEnd = Entities.offsetEnd,
   entities.confidence_score = Entities.confidence_score,
   entities.wikipediaExternalRef = Entities.wikipediaExternalRef,
   entities.wikidataId = Entities.wikidataId

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [378]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Cross-site scripting"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [379]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"XSS"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [380]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"vulnerabilities"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [381]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Untrusted"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [382]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"enters"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [383]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web application"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [384]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web request"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [385]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"2"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [386]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"dynamically generates"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [387]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web page"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [388]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web browser"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [389]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"JavaScript"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [390]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"HTML"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [391]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"tags"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [392]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"HTML attributes"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [393]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Flash"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [394]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"ActiveX"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [395]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"malicious"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [396]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"injected"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [397]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web server"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [398]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"executes"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [399]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"violates"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [400]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web browser's"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [401]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"same-origin policy"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [402]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"scripts"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [403]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Type 1"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [404]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Reflected XSS"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [405]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Persistent"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [406]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"HTTP request"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [407]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"HTTP response"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [408]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"exploits"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [409]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"attacker"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [410]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"parameter"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [411]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"URL"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [412]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"e-mailed"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [413]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"URLs"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [414]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"phishing"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [415]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"browser"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [416]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Type 2"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [417]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Stored XSS"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [418]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"visitor"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [419]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"data store"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [420]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"dynamic content"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [421]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"optimal"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [422]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"inject"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [423]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Interesting"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [424]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"privileges"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [425]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"sensitive data"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [426]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"privileged"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [427]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"gain access"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [428]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"log message"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [429]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"logs"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [430]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Type 0"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [431]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"DOM"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [432]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Based"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [433]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"performs"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [434]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"injection"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [435]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Javascript"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [436]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"sanity"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [437]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"injects"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [438]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"dynamic HTML"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [439]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"private information"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [440]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web site"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [441]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"administrator privileges"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [442]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Phishing attacks"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [443]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"emulate"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [444]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"web sites"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [445]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Finally"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [446]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"exploit"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [447]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"vulnerability"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [448]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"drive-by"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [449]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"hacking"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [450]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"attackers"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [451]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"variety of methods"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [452]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"encode"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [453]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"URL encoding"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [454]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"Unicode"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [455]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"submits"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [456]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"executable"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [457]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:2992})
MERGE (Entities:Entities {rawName:"untrusted"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [458]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"89"})
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.019997397708562056,source:"wikipedia-en",category:"Data-centric programming languages",page_id:925067}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009998698833835067,source:"wikipedia-en",category:"Command and control",page_id:13460558}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [459]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/Extended_Descriptions.json", null, {failOnError:false})
YIELD value
WITH value  AS data
WITH data
UNWIND data.Extended_Descriptions[1].CWE.entities as Entities
MERGE (entities:Entities{rawName:Entities.`rawName`})
  SET 
   entities.offsetStart = Entities.offsetStart,
   entities.offsetEnd = Entities.offsetEnd,
   entities.confidence_score = Entities.confidence_score,
   entities.wikipediaExternalRef = Entities.wikipediaExternalRef,
   entities.wikidataId = Entities.wikidataId

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [460]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities: Entities {rawName:"quoting"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [461]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName:"SQL syntax"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [462]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName:"controllable"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [463]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "inputs"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [464]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "SQL query"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [465]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "interpreted"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [466]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "SQL"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [467]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "user data"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [468]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "query"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [469]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "bypass"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [470]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "security checks"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [471]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "insert"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [472]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "modify"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [473]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "back-end database"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [474]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "commands"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [475]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "SQL injection"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [476]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "database-driven"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [477]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "web sites"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [478]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "flaw"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [479]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "exploited"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [480]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "software package"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [481]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime:691})
MERGE (Entities:Entities {rawName: "user base"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [482]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"94"})
MERGE (cwedata:Extended_Descriptions {runtime:833})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.0901926432383948,source:"wikipedia-en",category:"Computer security exploits",page_id:22272764}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.06764448218898413,source:"wikipedia-en",category:"Hacking (computer security)",page_id:11647669}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.022548160729661378,source:"wikipedia-en",category:"Programming language topics",page_id:5699671}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [483]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/Extended_Descriptions.json", null, {failOnError:false})
YIELD value
WITH value  AS data
WITH data
UNWIND data.Extended_Descriptions[2].CWE.entities as Entities
MERGE (entities:Entities{rawName:Entities.`rawName`})
  SET 
   entities.offsetStart = Entities.offsetStart,
   entities.offsetEnd = Entities.offsetEnd,
   entities.confidence_score = Entities.confidence_score,
   entities.wikipediaExternalRef = Entities.wikipediaExternalRef,
   entities.wikidataId = Entities.wikidataId

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [484]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "syntax"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [485]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "attacker"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [486]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "control flow"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [487]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "alteration"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [488]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "arbitrary code execution"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [489]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "Injection"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [490]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "wide variety"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [491]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "mitigated"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [492]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "weaknesses"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [493]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "classify"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [494]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "injection"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [495]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "control plane"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [496]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "controlled data"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [497]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "data channels"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [498]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "buffer overflows"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [499]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "flaws"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [500]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "parsed"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [501]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "instantiations"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [502]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "SQL injection"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [503]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 833})
MERGE (Entities:Entities {rawName: "format string vulnerabilities"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [504]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"200"})
MERGE (cwedata:Extended_Descriptions {runtime:1302})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009935460988123061,source:"wikipedia-en",category:"Construction",page_id:755979}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.009935460988123061,source:"wikipedia-en",category:"File sharing",page_id:802056}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [505]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/Extended_Descriptions.json", null, {failOnError:false})
YIELD value
WITH value  AS data
WITH data
UNWIND data.Extended_Descriptions[3].CWE.entities as Entities
MERGE (entities:Entities{rawName:Entities.`rawName`})
  SET 
   entities.offsetStart = Entities.offsetStart,
   entities.offsetEnd = Entities.offsetEnd,
   entities.confidence_score = Entities.confidence_score,
   entities.wikipediaExternalRef = Entities.wikipediaExternalRef,
   entities.wikidataId = Entities.wikidataId

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [506]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "exposures"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [507]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "severity"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [508]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "sensitive information"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [509]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "attacker"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [510]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "personal information"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [511]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "personal messages"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [512]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "financial data"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [513]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "health records"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [514]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "geographic location"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [515]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "contact details"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [516]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "packages"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [517]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "intellectual property"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [518]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "configuration"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [519]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "internal state"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [520]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "metadata"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [521]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "logging"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [522]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "headers"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [523]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "indirect"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [524]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "discrepancy"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [525]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "outsider"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [526]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "Information"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [527]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "direct product"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [528]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "administrators"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [529]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "admins"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [530]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "and/or"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [531]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "explicitly"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [532]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "inserts"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [533]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "unauthorized"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [534]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "script error"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [535]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1302})
MERGE (Entities:Entities {rawName: "revealing"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [536]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"284"})
MERGE (cwedata:Extended_Descriptions {runtime:1089})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.02177728820880549,source:"wikipedia-en",category:"Perimeter security",page_id:29568962}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.018774733450646813,source:"wikipedia-en",category:"Command and control",page_id:13460558}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.0054037453131323635,source:"wikipedia-en",category:"Software architecture",page_id:1805593}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [537]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/Extended_Descriptions.json", null, {failOnError:false})
YIELD value
WITH value  AS data
WITH data
UNWIND data.Extended_Descriptions[4].CWE.entities as Entities
MERGE (entities:Entities{rawName:Entities.`rawName`})
  SET 
   entities.offsetStart = Entities.offsetStart,
   entities.offsetEnd = Entities.offsetEnd,
   entities.confidence_score = Entities.confidence_score,
   entities.wikipediaExternalRef = Entities.wikipediaExternalRef,
   entities.wikidataId = Entities.wikidataId

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [538]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "Access control"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [539]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "Authentication"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [540]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "proving"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [541]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "Authorization"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [542]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "Accountability"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [543]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "attackers"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [544]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "privileges"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [545]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "sensitive information"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [546]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "executing"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [547]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "commands"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [548]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "evading detection"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [549]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "behaviors"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [550]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "access control"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [551]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "weaknesses"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [552]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "incorrect"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [553]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "permissions"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [554]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "explicitly"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [555]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "syntactically"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [556]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "ACL"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [557]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "insecure"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [558]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 1089})
MERGE (Entities:Entities {rawName: "security policy"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [559]:
explore_reviews_csv_query = """
MERGE (cwe:CWE {cwe_id:"294"})
MERGE (cwedata:Extended_Descriptions {runtime:253})
MERGE (cwe)-[:MAPPED_WITH]->(cwedata)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.09559506604831125,source:"wikipedia-en",category:"User interfaces",page_id:30875525}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.09559506604831125,source:"wikipedia-en",category:"Computing commands",page_id:28119161}]->(cwe)
MERGE (cwedata)-[:HAS_GLOBAL_CATEGORY{weight:0.09559506604831125,source:"wikipedia-en",category:"Computing terminology",page_id:2458835}]->(cwe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [560]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/Extended_Descriptions.json", null, {failOnError:false})
YIELD value
WITH value  AS data
WITH data
UNWIND data.Extended_Descriptions[5].CWE.entities as Entities
MERGE (entities:Entities{rawName:Entities.`rawName`})
  SET 
   entities.offsetStart = Entities.offsetStart,
   entities.offsetEnd = Entities.offsetEnd,
   entities.confidence_score = Entities.confidence_score,
   entities.wikipediaExternalRef = Entities.wikipediaExternalRef,
   entities.wikidataId = Entities.wikidataId

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [561]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 253})
MERGE (Entities:Entities {rawName: "Capture"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [562]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 253})
MERGE (Entities:Entities {rawName: "replay attacks"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [563]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 253})
MERGE (Entities:Entities {rawName: "cryptography"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [564]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 253})
MERGE (Entities:Entities {rawName: "subset"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [565]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 253})
MERGE (Entities:Entities {rawName: "injection"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [566]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 253})
MERGE (Entities:Entities {rawName: "observing"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result) 

[]


In [567]:
explore_reviews_csv_query = """
MERGE (cwedata:Extended_Descriptions {runtime: 253})
MERGE (Entities:Entities {rawName: "commands"})
MERGE (cwedata)-[:HAS_ENTITY]->(Entities)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **CPE Data**

**Script_CPE.cypher**

Parse CPE values in CVE using cpeparser

In [568]:
explore_reviews_csv_query = """
CALL apoc.load.json("https://raw.githubusercontent.com/neostrange/CveParse/main/CPE.json", null, {failOnError:false})
YIELD value
WITH value  AS data
//return data.CPE_Items[0].CPE.version
UNWIND data.CPE_Items as item
MERGE (cpe:CPE {version: item.CPE.version})
SET 
    cpe.part = item.CPE.part,
    cpe.vendor = item.CPE.vendor,
    cpe.product = item.CPE.product,
    cpe.update = item.CPE.update,
    cpe.edition = item.CPE.edition,
    cpe.language = item.CPE.language,
    cpe.sw_edition = item.CPE.sw_edition,
    cpe.target_sw = item.CPE.target_sw,
    cpe.target_hw = item.CPE.target_hw,
    cpe.other = item.CPE.other
    
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [569]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:host_agent:7.21:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.21"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [570]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:host_agent:7.22:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [571]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:o:microsoft:windows:-:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"-"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [572]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:702:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"702"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [573]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:750:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"750"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [574]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:752:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"752"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [575]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:753:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"753"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [576]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:754:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"754"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [577]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:755:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"755"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [578]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:756:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"756"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [579]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:731:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"731"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [580]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:740:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"740"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [581]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:751:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"751"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [582]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:757:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"757"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [583]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:702:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"702"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [584]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64nuc:7.22ext:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22ext"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [585]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64uc:7.53:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.53"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [586]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64nuc:7.22:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [587]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.22:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [588]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.53:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.53"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [589]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.77:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.77"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [590]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.81:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.81"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [591]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.85:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.85"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [592]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_krnl64uc:7.22ext:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.22ext"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [593]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:700:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"700"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [594]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:701:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"701"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [595]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:710:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"710"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [596]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:711:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"711"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [597]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:730:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"730"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [598]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:731:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"731"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [599]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:740:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"740"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [600]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:750:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"750"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [601]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:751:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"751"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [602]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:752:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"752"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [603]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:753:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"753"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [604]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:754:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"754"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [605]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:755:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"755"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [606]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:756:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"756"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [607]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_abap:757:*:*:*:sap_basis:*:*:*" })
MERGE (cpe:CPE {version:"757"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [608]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_as_abap_kernel:7.89:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.89"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [609]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:business_objects_business_intelligence_platform:420:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"420"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [610]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:business_planning_and_consolidation:810:*:*:*:*:microsoft:*:*" })
MERGE (cpe:CPE {version:"810"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [611]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:business_planning_and_consolidation:800:*:*:*:*:microsoft:*:*" })
MERGE (cpe:CPE {version:"800"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [612]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:netweaver_application_server_for_java:7.50:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"7.50"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [613]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:420:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"420"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [614]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:430:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"430"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [615]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:420:*:*:*:analysis:olap:*:*" })
MERGE (cpe:CPE {version:"420"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [616]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:businessobjects_business_intelligence_platform:430:*:*:*:analysis:olap:*:*" })
MERGE (cpe:CPE {version:"430"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [617]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:bank_account_management:900:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"900"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [618]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:sap:bank_account_management:800:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"800"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [619]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:a:linagora:twake:*:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"*"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [620]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:o:multilaserempresas:re708_firmware:re1200r4gc-2t2r-v3_v3411b_mul029b:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"re1200r4gc-2t2r-v3_v3411b_mul029b"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [621]:
explore_reviews_csv_query = """
MERGE (cpeMatch:CpeMatch {uri:"cpe:2.3:h:multilaserempresas:re708:-:*:*:*:*:*:*:*" })
MERGE (cpe:CPE {version:"-"})
MERGE (cpeMatch)-[:MAPPED_WITH]->(cpe)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


**Script_Unstructured data.cypher**

Extract the entities from unstructured data (vulnerability description) in CVE by using OIE model, entity-fishing, and en_core_web_sm model (from spaCy)


In [622]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"gains",type:"verb"}),
       (node2:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node3:Reference{text:"who",type:"R-ARG0"}),
       (node4:Frame_Argument{text:"local membership",type:"ARG1"}),
       (node5:Frame_Argument{text:"to SAP_LocalAdmin",type:"ARG4"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"gains"})
MERGE (description)-[:MAPPED_WITH]->(frame)

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [623]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"be",type:"verb"}),
       (node2:Frame_Argument{text:"an attacker who gains local membership to SAP_LocalAdmin",type:"ARG1"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"able to replace executables with a malicious file that will be started under a privileged account",type:"ARG2"}),
       (node5:Frame_Argument{type:"ARG0",Head:"attacker",offsetStart:3,offsetEnd:11,confidence_score:0.3651,wikipediaExternalRef:428681,wikidataId:"Q280658",domains:"Biology"}),
       (node6:Frame_Argument{type:"ARG2",Head:"malicious",offsetStart:35,offsetEnd:44,confidence_score:0.5011,wikipediaExternalRef:20901,wikidataId:"Q14001",domains:"Computer_Science"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1),
       (node6)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"be"})
MERGE (description)-[:MAPPED_WITH]->(frame)       
       
"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [624]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"replace",type:"verb"}),
       (node2:Frame_Argument{text:"executables",type:"ARG1"}),
       (node3:Frame_Argument{text:"an attacker who gains local membership to SAP_LocalAdmin",type:"ARG2"}),
       (node4:Frame_Argument{text:"with a malicious file that will be started under a privileged account",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"replace"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [625]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"started",type:"verb"}),
       (node2:Frame_Argument{text:"a malicious file",type:"ARG1"}),
       (node3:Reference{text:"that",type:"R-ARG1"}),
       (node4:Modifier_Modal{text:"will",type:"ARGM-MOD"}),
       (node5:Modifier_Location{text:"under a privileged account",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"started"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [626]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"Note",type:"verb"}),
       (node2:Modifier_Location{text:"In SAP",type:"ARGM-LOC"}),
       (node3:Frame_Argument{text:"that by default all user members of SAP_LocaAdmin are denied the ability to logon locally by security policy so that this can only occur if the system has already been compromised",type:"ARG1"}),
       (node4:Frame_Argument{type:"ARG1",Head:"logon",offsetStart:76,offsetEnd:81,confidence_score:0.6351,wikipediaExternalRef:9878372,wikidataId:"Q472302",domains:"Computer_Science"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"Note"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [627]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"denied",type:"Verb"}),
       (node2:Modifier_Manner{text:"by default all user members of",type:"ARGM-MNR"}),
       (node3:Frame_Argument{text:"the ability to logon locally by security policy",type:"ARG1"}),
       (node4:Modifier_Purpose{text:"so that this can only occur if the system has already been compromised",type:"ARGM-PRP"}),
       (node5:Frame_Argument{type:"ARG0",Head:"security policy",offsetStart:32,offsetEnd:47,confidence_score:0.5825,wikipediaExternalRef:34231217,wikidataId:"Q1128636",domains:"Electronics, Computer_Science, Telecommunication"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"denied"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [628]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"logon",type:"Verb"}),
       (node2:Frame_Argument{text:"locally",type:"ARG1"}),
       (node3:Frame_Argument{text:"by security",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"logon"})
MERGE (description)-[:MAPPED_WITH]->(frame)       
      

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [629]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"occur",type:"Verb"}),
       (node2:Frame_Argument{text:"this",type:"ARG1"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Modifier_Adverbial{text:"only",type:"ARGM-ADV"}),
       (node5:Modifier_Adverbial{text:"if the system has already been compromised",type:"ARGM-ADV"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"occur"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [630]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"compromised",type:"Verb"}),
       (node2:Frame_Argument{text:"the system",type:"ARG1"}),
       (node3:Modifier_Temporal{text:"already",type:"ARGM-TMP"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0012"})
MERGE (frame:Frame {text:"compromised"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [631]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"encode",type:"Verb"}),
       (node2:Frame_Argument{text:"The ABAP Keyword Documentation of SAP NetWeaver Application Server - versions 702 , 731 , 740 , 750 , 751 , 752 , 753 , 754 , 755 , 756 , 757 , for ABAP and ABAP Platform",type:"ARG0"}),
       (node3:Modifier_Negation{text:"not",type:"ARGM-NEG"}),
       (node4:Modifier_Manner{text:"sufficiently",type:"ARGM-MNR"}),
       (node5:Frame_Argument{text:"user - controlled inputs",type:"ARG1"}),
       (node6:Modifier_Adverbial{text:"resulting in Cross - Site Scripting ( XSS ) vulnerability . On successful exploitation an attacker can cause limited impact on confidentiality and integrity of the application",type:"ARGM-ADV"}),
       (node7:Frame_Argument{type:"ARG1",Head:"user-controlled",offsetStart:189,offsetEnd:204,confidence_score:0.4601,wikipediaExternalRef:21875249,wikidataId:"Q3561045",domains:"Computer_Science"}),
       (node8:Frame_Argument{type:"ARG1",Head:"inputs",offsetStart:205,offsetEnd:211,confidence_score:0.3879,wikipediaExternalRef:11231,wikidataId:"Q210947",domains:"Finance,Sociology,Economy,Commerce,Enterprise"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1),
       (node6)-[:HAS_ARGUMENT]->(node1),
       (node7)-[:HAS_ARGUMENT]->(node1),
       (node8)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0013"})
MERGE (frame:Frame {text:"encode"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [632]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"controlled",type:"Verb"}),
       (node2:Frame_Argument{text:"user",type:"ARG0"}),
       (node3:Frame_Argument{text:"inputs",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0013"})
MERGE (frame:Frame {text:"controlled"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [633]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"cause",type:"Verb"}),
       (node2:Modifier_Location{text:"On successful exploitation",type:"ARGM-LOC"}),
       (node3:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"limited impact on confidentiality and integrity of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0013"})
MERGE (frame:Frame {text:"cause"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [634]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"creates",type:"Verb"}),
       (node2:Frame_Argument{text:"SAP NetWeaver ABAP Server and ABAP Platform - versions SAP_BASIS 700 , 701 , 702 , 710 , 711 , 730 , 731 , 740 , 750 , 751 , 752 , 753 , 754 , 755 , 756 , 757 , KERNEL 7.22 , 7.53 , 7.77 , 7.81 , 7.85 , 7.89 , KRNL64UC 7.22 , 7.22EXT , 7.53 , KRNL64NUC 7.22 , 7.22EXT",type:"ARG0"}),
       (node3:Frame_Argument{text:"information about system identity",type:"ARG1"}),
       (node4:Modifier_Location{text:"in an ambiguous format",type:"ARGM-LOC"}),
       (node5:Frame_Argument{type:"ARG1",Head:"KERNEL",offsetStart:161,offsetEnd:167,confidence_score:0.4157,wikipediaExternalRef:21346982,wikidataId:"Q9662",domains:"Computer_Science"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"creates"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [635]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"lead",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to capture - replay vulnerability",type:"ARG2"}),
       (node5:Frame_Argument{type:"ARG1",Head:"vulnerability",offsetStart:20,offsetEnd:33,confidence_score:0.3654,wikipediaExternalRef:1129827,wikidataId:"Q631425",domains:"Military,Computer_Science"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"lead"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [636]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"may",type:"Verb"}),
       (node2:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node3:Frame_Argument{text:"be exploited by malicious users to obtain illegitimate access to the system",type:"ARG1"}),
       (node4:Frame_Argument{type:"ARG0",Head:"malicious users",offsetStart:16,offsetEnd:31,confidence_score:0.6062,wikipediaExternalRef:2471540,wikidataId:"Q2798820",domains:"Computer_Science"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"may"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [637]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"exploited",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG1"}),
       (node3:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"by malicious users",type:"ARG0"}),
       (node5:Frame_Argument{text:"to obtain illegitimate access to the system",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0014"})
MERGE (frame:Frame {text:"exploited"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [638]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"return",type:"Verb"}),
       (node2:Frame_Argument{text:"some calls",type:"ARG0"}),
       (node3:Frame_Argument{text:"json",type:"ARG1"}),
       (node4:Modifier_Manner{text:"with wrong content type",type:"ARGM-MNR"}),
       (node5:Modifier_Manner{text:"in the header of the",type:"ARGM-MNR"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"return"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [639]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"calls",type:"Verb"}),
       (node2:Frame_Argument{text:"a custom application",type:"ARG0"}),
       (node3:Reference{text:"that",type:"R-ARG0"}),
       (node4:Modifier_Manner{text:"directly",type:"ARGM-MNR"}),
       (node5:Frame_Argument{text:"the jsp of Web Intelligence",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"calls"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [640]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"be",type:"Verb"}),
       (node2:Frame_Argument{text:"a custom application that calls directly the jsp of Web Intelligence DHTML",type:"ARG1"}),
       (node3:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"vulnerable to XSS attacks",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"be"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [641]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"cause",type:"Verb"}),
       (node2:Modifier_Location{text:"On successful exploitation",type:"ARGM-LOC"}),
       (node3:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"limited impact on confidentiality and integrity of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0015"})
MERGE (frame:Frame {text:"cause"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [642]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allows",type:"Verb"}),
       (node2:Frame_Argument{text:"SAP BPC MS 10.0 - version 810",type:"ARG0"}),
       (node3:Frame_Argument{text:"an unauthorized attacker to execute crafted database queries",type:"ARG1"}),
       (node4:Frame_Argument{type:"ARG1",Head:"unauthorized",offsetStart:41,offsetEnd:53,confidence_score:0.3687,wikipediaExternalRef:151617,wikidataId:"Q788590",domains:"Sociology,Electronics,Engineering"}),
       (node5:Frame_Argument{type:"ARG1",Head:"attacker",offsetStart:54,offsetEnd:62,confidence_score:0.4001,wikipediaExternalRef:428681,wikidataId:"Q280658",domains:"Biology"}),


       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"allows"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [643]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"execute",type:"Verb"}),
       (node2:Frame_Argument{text:"an unauthorized attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"crafted database queries",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"execute"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [644]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"lead",type:"Verb"}),
       (node2:Frame_Argument{text:"The exploitation of this issue",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to SQL injection vulnerability",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"lead"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [645]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"could",type:"Verb"}),
       (node2:Frame_Argument{text:"The exploitation of this issue",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"allow an attacker to access , modify , and/or delete data from the backend database",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"could"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [646]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allow",type:"Verb"}),
       (node2:Frame_Argument{text:"The exploitation of this issue",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"an attacker to access , modify , and/or delete data from the backend database",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0016"})
MERGE (frame:Frame {text:"allow"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [647]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"can",type:"Verb"}),
       (node2:Frame_Argument{text:"An unauthenticated attacker in SAP NetWeaver AS for Java - version 7.50",type:"ARG0"}),
       (node3:Frame_Argument{text:"attach to an open interface and make use of an open naming and directory API to access services which can be used to perform unauthorized operations affecting users and data on the current",type:"ARG1"}),
       (node4:Frame_Argument{type:"ARG1",Head:"unauthorized",offsetStart:125,offsetEnd:137,confidence_score:0.3803,wikipediaExternalRef:18948365,wikidataId:"Q647578",domains:"Electronics,Computer_Science,Economy,Sociology"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"can"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [648]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"attach",type:"Verb"}),
       (node2:Frame_Argument{text:"An unauthenticated attacker in SAP NetWeaver AS for Java - version 7.50 , due to improper access control",type:"ARG0"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to an open interface",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"attach"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [649]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"make",type:"Verb"}),
       (node2:Frame_Argument{text:"An unauthenticated attacker in SAP NetWeaver AS for Java - version 7.50 , due to improper access control",type:"ARG0"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"use of an open naming and directory API to access services which can be used to perform unauthorized operations affecting users and data on the current system",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"make"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [650]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"used",type:"Verb"}),
       (node2:Frame_Argument{text:"access services",type:"ARG1"}),
       (node3:Reference{text:"which",type:"R-ARG1"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"to perform unauthorized operations affecting users and data on the current system",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"used"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [651]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"perform",type:"Verb"}),
       (node2:Frame_Argument{text:"access services",type:"ARG0"}),
       (node3:Frame_Argument{text:"unauthorized operations affecting users and data on the current system",type:"ARG1"}),
       (node4:Frame_Argument{type:"ARG1",Head:"unauthorized",offsetStart:234,offsetEnd:246,confidence_score:0.3753,wikipediaExternalRef:205662,wikidataId:"Q502918",domains:"Commerce, Enterprise, Finance"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"perform"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [652]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"affecting",type:"Verb"}),
       (node2:Frame_Argument{text:"unauthorized operations",type:"ARG0"}),
       (node3:Frame_Argument{text:"users",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"affecting"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [653]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"could",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG0"}),
       (node3:Frame_Argument{text:"allow the attacker to have full read access to user data , make modifications to user data , and make services within the system unavailable",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"could"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [654]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allow",type:"Verb"}),
       (node2:Frame_Argument{text:"This",type:"ARG0"}),
       (node3:Modifier_Modal{text:"could",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"the attacker to have full read access to user data , make modifications to user data , and make services within the system unavailable",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"allow"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [655]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"have",type:"Verb"}),
       (node2:Frame_Argument{text:"the attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"full read access to user data",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"have"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [656]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"user",type:"Verb"}),
       (node2:Frame_Argument{text:"full read",type:"ARG0"}),
       (node3:Frame_Argument{text:"data",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"user"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [657]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"make",type:"Verb"}),
       (node2:Frame_Argument{text:"the attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"modifications to user data , and make",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"make"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [658]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"make",type:"Verb"}),
       (node2:Frame_Argument{text:"the attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"services within the system",type:"ARG1"}),
       (node4:Modifier_Manner{text:"unavailable",type:"ARGM-MNR"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0017"})
MERGE (frame:Frame {text:"make"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [659]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"controlled",type:"Verb"}),
       (node2:Frame_Argument{text:"user",type:"ARG0"}),
       (node3:Frame_Argument{text:"input",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"controlled"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [660]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"modify",type:"Verb"}),
       (node2:Modifier_Cause{text:"Due to improper input sanitization of user - controlled input in SAP BusinessObjects Business Intelligence",type:"ARGM-CAU"}),
       (node3:Frame_Argument{text:"an attacker with basic user - level privileges",type:"ARG0"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"reports containing a malicious payload",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)
       
MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"modify"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [661]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"containing",type:"Verb"}),
       (node2:Frame_Argument{text:"reports",type:"ARG0"}),
       (node3:Frame_Argument{text:"a malicious payload",type:"ARG1"}),
       
       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)
      
MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"containing"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [662]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"are",type:"Verb"}),
       (node2:Frame_Argument{text:"these reports",type:"ARG1"}),
       (node3:Frame_Argument{text:"viewable",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"are"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [663]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"opens",type:"Verb"}),
       (node2:Frame_Argument{text:"anyone",type:"ARG0"}),
       (node3:Reference{text:"who",type:"R-ARG0"}),
       (node4:Frame_Argument{text:"those reports",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"opens"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [664]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"be",type:"Verb"}),
       (node2:Frame_Argument{text:"anyone who opens those reports",type:"ARG1"}),
       (node3:Modifier_Modal{text:"would",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"susceptible to stored XSS attacks",type:"ARG2"}),
       (node5:Modifier_Cause{text:"As",type:"ARGM-CAU"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"be"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [665]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"maintained",type:"Verb"}),
       (node2:Frame_Argument{text:"information",type:"ARG1"}),
       (node3:Modifier_Location{text:"in the victim 's web browser",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"maintained"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [666]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"read",type:"Verb"}),
       (node2:Modifier_Cause{text:"As a result of the attack",type:"ARGM-CAU"}),
       (node3:Frame_Argument{text:"information maintained in the victim 's web browser",type:"ARG1"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"read"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [667]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"modified",type:"Verb"}),
       (node2:Frame_Argument{text:"information maintained in the victim 's web browser",type:"ARG1"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"modified"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [668]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"sent",type:"Verb"}),
       (node2:Frame_Argument{text:"information maintained in the victim 's web browser",type:"ARG1"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"to the attacker",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0018"})
MERGE (frame:Frame {text:"sent"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [669]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"allows",type:"Verb"}),
       (node2:Frame_Argument{text:"SAP BusinessObjects Business Intelligence Analysis edition for OLAP",type:"ARG0"}),
       (node3:Frame_Argument{text:"an authenticated attacker to inject malicious code that can be executed by the application over the network . On successful exploitation , an attacker can perform operations that may completely compromise the application causing a high impact on the confidentiality , integrity , and availability of the application",type:"ARG1"}),
       (node4:Frame_Argument{type:"ARG0",Head:"authenticated",offsetStart:3,offsetEnd:16,confidence_score:0.3605,wikipediaExternalRef:47967,wikidataId:"Q212108",domains:"Economy,Electronics,Engineering"}),
       (node5:Frame_Argument{type:"ARG1",Head:"malicious code",offsetStart:36,offsetEnd:50,confidence_score:0.4342,wikipediaExternalRef:20901,wikidataId:"Q14001",domains:"Computer_Science"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"allows"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [670]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"inject",type:"Verb"}),
       (node2:Frame_Argument{text:"an authenticated attacker",type:"ARG0"}),
       (node3:Frame_Argument{text:"malicious code that can be executed by the application over the network . On successful exploitation",type:"ARG1"}),
       (node4:Frame_Argument{type:"ARG1",Head:"malicious code",offsetStart:0,offsetEnd:14,confidence_score:0.4342,wikipediaExternalRef:20901,wikidataId:"Q14001",domains:"Computer_Science"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"inject"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [671]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"executed",type:"Verb"}),
       (node2:Frame_Argument{text:"malicious code",type:"ARG1"}),
       (node3:Reference{text:"that",type:"R-ARG1"}),
       (node4:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node5:Frame_Argument{text:"by the application over the network",type:"ARG0"}),
      
       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"executed"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [672]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"perform",type:"Verb"}),
       (node2:Frame_Argument{text:"an attacker",type:"ARG0"}),
       (node3:Modifier_Modal{text:"can",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"operations",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"perform"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [673]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"compromise",type:"Verb"}),
       (node2:Frame_Argument{text:"operations",type:"ARG0"}),
       (node3:Reference{text:"that",type:"R-ARG0"}),
       (node4:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node5:Modifier_Extent{text:"completely",type:"ARGM-EXT"}),
       (node6:Frame_Argument{text:"the application causing a high impact on the confidentiality , integrity , and availability of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1),
       (node5)-[:HAS_ARGUMENT]->(node1),
       (node6)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"compromise"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [674]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"causing",type:"Verb"}),
       (node2:Frame_Argument{text:"the application",type:"ARG0"}),
       (node3:Frame_Argument{text:"a high impact on the confidentiality , integrity , and availability of the application",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0022"})
MERGE (frame:Frame {text:"causing"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [675]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"clicks",type:"Verb"}),
       (node2:Modifier_Temporal{text:"when",type:"ARGM-TMP"}),
       (node3:Frame_Argument{text:"a user",type:"ARG0"}),
       (node4:Frame_Argument{text:"a smart link to navigate to another app",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"clicks"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [676]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"navigate",type:"Verb"}),
       (node2:Frame_Argument{text:"a user",type:"ARG0"}),
       (node3:Frame_Argument{text:"to another app",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"navigate"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [677]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"shown",type:"Verb"}),
       (node2:Modifier_Location{text:"In SAP Bank Account Management ( Manage Banks ) application",type:"ARGM-LOC"}),
       (node3:Modifier_Temporal{text:"when a user clicks a smart link to navigate to another app",type:"ARGM-TMP"}),
       (node4:Frame_Argument{text:"personal data",type:"ARG1"}),
       (node5:Modifier_Manner{text:"directly",type:"ARGM-MNR"}),
       (node6:Modifier_Location{text:"in the URL",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1), 
       (node5)-[:HAS_ARGUMENT]->(node1),
       (node6)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"shown"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [678]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"get",type:"Verb"}),
       (node2:Frame_Argument{text:"They",type:"ARG1"}),
       (node3:Modifier_Modal{text:"might",type:"ARGM-MOD"}),
       (node4:Frame_Argument{text:"captured in log files , bookmarks , and so on disclosing sensitive data of the application",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"get"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [679]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"captured",type:"Verb"}),
       (node2:Frame_Argument{text:"They",type:"ARG1"}),
       (node3:Modifier_Modal{text:"might",type:"ARGM-MOD"}),
       (node4:Modifier_Location{text:"in log files , bookmarks",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0023"})
MERGE (frame:Frame {text:"captured"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [680]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"Stored",type:"Verb"}),
       (node2:Frame_Argument{text:")",type:"ARG0"}),
       (node3:Frame_Argument{text:"in GitHub repository linagora / twake",type:"ARG1"}),
       (node4:Modifier_Temporal{text:"prior to 2023.Q1.1200 +",type:"ARGM-TMP"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0028"})
MERGE (frame:Frame {text:"Stored"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [681]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"found",type:"Verb"}),
       (node2:Frame_Argument{text:"A vulnerability",type:"ARG1"}),
       (node3:Modifier_Location{text:"in Multilaser RE708 RE1200R4GC-2T2R - V3_v3411b_MUL029B.",type:"ARGM-LOC"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"found"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [682]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"rated",type:"Verb"}),
       (node2:Frame_Argument{text:"in Multilaser RE708 RE1200R4GC-2T2R - V3_v3411b_MUL029B",type:"ARG2"}),
       (node3:Frame_Argument{text:"It",type:"ARG1"}),
       (node4:Frame_Argument{text:"as problematic",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"rated"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [683]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"affects",type:"Verb"}),
       (node2:Frame_Argument{text:"This issue",type:"ARG0"}),
       (node3:Frame_Argument{text:"some unknown processing of the component Telnet Service",type:"ARG1"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"affects"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [684]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"leads",type:"Verb"}),
       (node2:Frame_Argument{text:"The manipulation",type:"ARG0"}),
       (node3:Frame_Argument{text:"to denial of service",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"leads"})
MERGE (description)-[:MAPPED_WITH]->(frame)        

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [685]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"initiated",type:"Verb"}),
       (node2:Frame_Argument{text:"The attack",type:"ARG1"}),
       (node3:Modifier_Modal{text:"may",type:"ARGM-MOD"}),
       (node4:Modifier_Manner{text:"remotely",type:"ARGM-MNR"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1),
       (node4)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"initiated"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


In [686]:
explore_reviews_csv_query = """
CREATE (node1:Frame {text:"assigned",type:"Verb"}),
       (node2:Frame_Argument{text:"The identifier VDB-217169",type:"ARG1"}),
       (node3:Frame_Argument{text:"to this vulnerability",type:"ARG2"}),

       (node2)-[:HAS_ARGUMENT]->(node1),
       (node3)-[:HAS_ARGUMENT]->(node1)

MERGE (description:Description {cveId:"CVE-2023-0029"})
MERGE (frame:Frame {text:"assigned"})
MERGE (description)-[:MAPPED_WITH]->(frame)       

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[]


# **Creating rules for enhancing the coverage of the knowledge graph**

Here all CWE refered to weaknesses that lead to CVE vulnerabilities 

In [687]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)
MERGE(cwe)-[:LEAD_TO]->(cve)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:6' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0018'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:155' labels=frozenset({'CWE'}) properties={'weakness_abstraction': 'Base', 'cwe_id': '79', 'name': "Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting')", 'description': 'The software does not neutralize or incorrectly neutralizes user-controllable input before it is placed in output that is used as a web page that is served to other users.', 'extended_description': "Cross-site scripting (XSS) vulnerabilities occur when: 1. Untrusted data enters a web application, typically from a web request. 2. The web application dynamically generates a web page that contains this untrusted data. 3. During page generation, the application does not prevent the data from containing content that is executable by a web browser, such as JavaScr

These patterns matchs some entities from CWE, that vulnerable to each CVE vulnerabilities

In [688]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{rawName:"Cross-site scripting"})
MERGE(Entities)<-[:VULNERABLE_TO]-(cve)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:6' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0018'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:167' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 241154, 'wikidataId': 'Q371199', 'confidence_score': 0.719, 'offsetStart': 0, 'offsetEnd': 20, 'rawName': 'Cross-site scripting'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:9' labels=frozenset({'CVE'}) properties={'assigner': 'security@huntr.dev', 'id': 'CVE-2023-0028'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:167' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 241154, 'wikidataId': 'Q371199', 'confidence_score': 0.719, 'offsetStart': 0, 'offsetEnd': 20, 'rawName': 'Cross-site scripting'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:3' labels=frozenset({'C

In [689]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{rawName:"SQL injection"})
MERGE(Entities)<-[:VULNERABLE_TO]-(cve)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:4' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0016'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:262' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 526999, 'wikidataId': 'Q506059', 'confidence_score': 0.9162, 'offsetStart': 1041, 'offsetEnd': 1054, 'rawName': 'SQL injection'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:7' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0022'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:262' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 526999, 'wikidataId': 'Q506059', 'confidence_score': 0.9162, 'offsetStart': 1041, 'offsetEnd': 1054, 'rawName': 'SQL injection'}> size=4>>]


In [690]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{rawName:"sensitive information"})
MERGE(Entities)<-[:VULNERABLE_TO]-(cve)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:8' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0023'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:287' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 5316476, 'wikidataId': 'Q2587068', 'confidence_score': 0.4076, 'offsetStart': 394, 'offsetEnd': 415, 'rawName': 'sensitive information'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:0' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0012'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:287' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 5316476, 'wikidataId': 'Q2587068', 'confidence_score': 0.4076, 'offsetStart': 394, 'offsetEnd': 415, 'rawName': 'sensitive information'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:5' labels=frozen

In [691]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{rawName:"Access control"})
MERGE(Entities)<-[:VULNERABLE_TO]-(cve)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:0' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0012'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:314' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 40684, 'wikidataId': 'Q228502', 'confidence_score': 0.6394, 'offsetStart': 0, 'offsetEnd': 14, 'rawName': 'Access control'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:5' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0017'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:314' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 40684, 'wikidataId': 'Q228502', 'confidence_score': 0.6394, 'offsetStart': 0, 'offsetEnd': 14, 'rawName': 'Access control'}> size=4>>]


In [692]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{rawName:"replay attacks"})
MERGE(Entities)<-[:VULNERABLE_TO]-(cve)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:2' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0014'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:331' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 614094, 'wikidataId': 'Q1756025', 'confidence_score': 0.5922, 'offsetStart': 8, 'offsetEnd': 22, 'rawName': 'replay attacks'}> size=4>>]


Here some CVE are connecting with wikidata Id of the entities in order to create similarities between CVEs

In [693]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{wikidataId:"Q371199"})
MERGE (cve1:CVE{id:"CVE-2023-0013"})
MERGE (cve2:CVE{id:"CVE-2023-0015"})
MERGE (cve3:CVE{id:"CVE-2023-0018"})
MERGE (cve4:CVE{id:"CVE-2023-0028"})
MERGE (cve1)-[:SIMILAR_TO]->(cve2)
MERGE (cve2)-[:SIMILAR_TO]->(cve3)
MERGE (cve3)-[:SIMILAR_TO]->(cve4)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:6' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0018'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:167' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 241154, 'wikidataId': 'Q371199', 'confidence_score': 0.719, 'offsetStart': 0, 'offsetEnd': 20, 'rawName': 'Cross-site scripting'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:6' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0018'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:168' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 241154, 'wikidataId': 'Q371199', 'confidence_score': 0.8255, 'offsetStart': 3130, 'offsetEnd': 3133, 'rawName': 'XSS'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:6' labels=frozenset({'CVE'}) properties={

In [694]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{wikidataId:"Q506059"})
MERGE (cve1:CVE{id:"CVE-2023-0016"})
MERGE (cve2:CVE{id:"CVE-2023-0022"})
MERGE (cve1)-[:SIMILAR_TO]->(cve2)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:4' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0016'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:262' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 526999, 'wikidataId': 'Q506059', 'confidence_score': 0.9162, 'offsetStart': 1041, 'offsetEnd': 1054, 'rawName': 'SQL injection'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:7' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0022'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:262' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 526999, 'wikidataId': 'Q506059', 'confidence_score': 0.9162, 'offsetStart': 1041, 'offsetEnd': 1054, 'rawName': 'SQL injection'}> size=4>>]


In [695]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{wikidataId:"Q2587068"})
MERGE (cve1:CVE{id:"CVE-2023-0012"})
MERGE (cve2:CVE{id:"CVE-2023-0017"})
MERGE (cve3:CVE{id:"CVE-2023-0023"})
MERGE (cve1)-[:SIMILAR_TO]->(cve2)
MERGE (cve2)-[:SIMILAR_TO]->(cve3)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:6' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0018'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:216' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 5316476, 'wikidataId': 'Q2587068', 'confidence_score': 0.4691, 'offsetStart': 2476, 'offsetEnd': 2490, 'rawName': 'sensitive data'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:9' labels=frozenset({'CVE'}) properties={'assigner': 'security@huntr.dev', 'id': 'CVE-2023-0028'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:216' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 5316476, 'wikidataId': 'Q2587068', 'confidence_score': 0.4691, 'offsetStart': 2476, 'offsetEnd': 2490, 'rawName': 'sensitive data'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:3' labels=frozenset

In [696]:
explore_reviews_csv_query = """
MATCH p=(cve:CVE)-[:PROBLEM_TYPE]->(problemType:ProblemType)-[:MAPPED_WITH]->(cwe:CWE)-[:MAPPED_WITH]->(cwedata:Extended_Descriptions)-[:HAS_ENTITY]->(Entities:Entities{wikidataId:"Q228502"})
MERGE (cve1:CVE{id:"CVE-2023-0012"})
MERGE (cve2:CVE{id:"CVE-2023-0017"})
MERGE (cve1)-[:SIMILAR_TO]->(cve2)
RETURN p

"""
result = conn.query(explore_reviews_csv_query)
print(result)

[<Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:0' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0012'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:322' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 40684, 'wikidataId': 'Q228502', 'confidence_score': 0.4556, 'offsetStart': 947, 'offsetEnd': 961, 'rawName': 'access control'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:0' labels=frozenset({'CVE'}) properties={'assigner': 'cna@sap.com', 'id': 'CVE-2023-0012'}> end=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:314' labels=frozenset({'Entities'}) properties={'wikipediaExternalRef': 40684, 'wikidataId': 'Q228502', 'confidence_score': 0.6394, 'offsetStart': 0, 'offsetEnd': 14, 'rawName': 'Access control'}> size=4>>, <Record p=<Path start=<Node element_id='4:de342d72-e7ff-4d5b-9a28-99eaa381965b:5' labels=frozenset({'CVE'}) properties